In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/KoELECTRA/finetune

/content/drive/MyDrive/KoELECTRA/finetune


In [ ]:
class Hyperparameters:
    def __init__(self):
        self.task = "hate-speech"
        self.data_dir = "data"
        self.ckpt_dir = "ckpt"
        self.train_file = "train.tsv"
        self.dev_file = "" #"dev.csv"
        self.test_file = "val.tsv"
        self.evaluate_test_during_training = True
        self.eval_all_checkpoints = True
        self.save_optimizer = False
        self.do_lower_case = False
        self.do_train = True
        self.do_eval = True
        self.max_seq_len = 128
        self.num_train_epochs = 10
        self.weight_decay = 0.0
        self.gradient_accumulation_steps = 1
        self.adam_epsilon = 1e-8
        self.warmup_proportion = 0
        self.max_steps = -1
        self.max_grad_norm = 1.0
        self.no_cuda = False
        self.model_type = "koelectra-base-v3"
        self.model_name_or_path = "monologg/koelectra-base-v3-discriminator"
        self.output_dir = "koelectra-base-v3-hate-speech-ckpt"
        self.seed = 42
        self.train_batch_size = 32
        self.eval_batch_size = 64
        self.logging_steps = 100
        self.save_steps = 100
        self.learning_rate = 5e-5
        self.device = ""


args = Hyperparameters()
print(args.task)  # 출력: "hate-speech"


hate-speech


In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=a8d0bbcbe60cdf56b5124c4b3d4ec8d4f41ccd8611a7d59aee007971afefcf60
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
import json
import logging
import os
import glob
import re

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)

from src import (
    CONFIG_CLASSES,
    TOKENIZER_CLASSES,
    MODEL_FOR_SEQUENCE_CLASSIFICATION,
    init_logger,
    set_seed,
    compute_metrics
)
from processor import seq_cls_load_and_cache_examples as load_and_cache_examples
from processor import seq_cls_tasks_num_labels as tasks_num_labels
from processor import seq_cls_processors as processors
from processor import seq_cls_output_modes as output_modes

logger = logging.getLogger(__name__)


In [ ]:
tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )

In [ ]:
global_step = 6600
output_dir = "/content/drive/MyDrive/KoELECTRA/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt"
ckpt_path = os.path.join(output_dir, f"checkpoint-{global_step}" )


In [ ]:
tokenizer.save_pretrained(ckpt_path)

('/content/drive/MyDrive/KoELECTRA/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt/checkpoint-6600/tokenizer_config.json',
 '/content/drive/MyDrive/KoELECTRA/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt/checkpoint-6600/special_tokens_map.json',
 '/content/drive/MyDrive/KoELECTRA/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt/checkpoint-6600/vocab.txt',
 '/content/drive/MyDrive/KoELECTRA/finetune/ckpt/koelectra-base-v3-hate-speech-ckpt/checkpoint-6600/added_tokens.json')

In [ ]:
!pip install transformers
!huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) ㅛ
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) ㅛ
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
A

In [ ]:
!huggingface-cli repo create koELECTRA_hatespeech_v2


y
git version 2.25.1
git-lfs/2.9.2 (GitHub; linux amd64; go 1.13.5)

You are about to create Haaaaeun/koELECTRA_hatespeech_v2
Proceed? [Y/n] 
Your repo now lives at:
  https://huggingface.co/Haaaaeun/koELECTRA_hatespeech_v2

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/Haaaaeun/koELECTRA_hatespeech_v2



In [ ]:
# !git clone https://huggingface.co/Haaaaeun/koELECTRA_hatespeech

Cloning into 'koELECTRA_hatespeech'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 414 bytes | 20.00 KiB/s, done.


In [ ]:
# %cd /content/drive/MyDrive/koELECTRA_hatespeech

/content/drive/MyDrive/koELECTRA_hatespeech


In [ ]:
# !git add --all

In [ ]:
# !git commit -m "upload ckpt 2100"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
# !git push

fatal: could not read Username for 'https://huggingface.co': No such device or address


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path=ckpt_path,
    repo_id="Haaaaeun/koELECTRA_hatespeech_v2",
    repo_type="model",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

'https://huggingface.co/Haaaaeun/koELECTRA_hatespeech_v2/tree/main/'